In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [109]:
import pandas as pd
df= pd.read_csv(r"/content/drive/MyDrive/Tweet_NFT.csv")

In [111]:
df.head()

,id,tweet_text,tweet_created_at,tweet_intent
0,1212762,@crypto_brody @eCoLoGy1990 @MoonrunnersNFT @It...,2022-08-06T16:56:36.000Z,Community
1,1212763,Need Sick Character artâ“#art #artist #Artist...,2022-08-06T16:56:36.000Z,Giveaway
2,1212765,@The_Hulk_NFT @INagotchiNFT @Tesla @killabears...,2022-08-06T16:56:35.000Z,Appreciation
3,1212766,@CryptoBatzNFT @DarekBTW The first project in ...,2022-08-06T16:56:35.000Z,Community
4,1212767,@sashadysonn The first project in crypto with ...,2022-08-06T16:56:34.000Z,Community


In [112]:
df.shape

(127453, 4)

In [113]:
#finding the null values in the dataframe
df.isna().sum()

id                      0
tweet_text              0
tweet_created_at        0
tweet_intent        31089
dtype: int64

In [114]:
#separating the tweets whose intent needs to be predicted to the dataframe tar
# tar is the test dataset
tar=df[df['tweet_intent'].isna()]

In [116]:
tar.shape

(31089, 4)

In [117]:
#dropping the columns with null values from df
dff=df.dropna()

In [118]:
dff.shape

(96364, 4)

In [119]:
96364+31089

127453

In [120]:
dff.nunique()

id                  96364
tweet_text          84908
tweet_created_at    47334
tweet_intent            9
dtype: int64

In [121]:
#dff is the training and validation dataset

In [123]:
(dff['tweet_text'][9])

'@SpaceBrosBSC PROJECT 21 - THE BEST GAMEFI PROJECT ON BSC\nâœ…Multistage deflation tool - PRO21\nâœ…Multi-feature NFT\nâœ…App on AppStore and Google Play\nâœ…Presale in August.\n\n$P21 IS GOING TO THE MOON!\nWhitelist: https://t.co/DMH92u9tNA\nTG: https://t.co/jPGNFcUQKE\n#Project21 #P21 #Gamefi'

In [124]:
#addding all the tweet_text to a list
twe_lis=[]
for i in dff['tweet_text']:
    twe_lis.append(i)

In [125]:
#remove 1.links 2.mentions 3.hashtags 4.punctuations
import re
text="@SpaceBrosBSC PROJECT 21 - THE BEST GAMEFI PROJECT ON BSC\nâœ…Multistage deflation tool - PRO21\nâœ…Multi-feature NFT\nâœ…App on AppStore and Google Play\nâœ…Presale in August.\n\n$P21 IS GOING TO THE MOON!\nWhitelist: https://t.co/DMH92u9tNA\nTG: https://t.co/jPGNFcUQKE\n#Project21 #P21 #Gamefi"
text = re.sub("https?:\/\/.*[\r\n]*", "", text)
text = re.sub("@\S+", "", text)
text = re.sub("#", "", text)

In [126]:
#removing \n from the sentences
text = text.replace('\n',"")

In [127]:
#removing all the unrequired characters
text = re.sub(r'[^a-zA-Z0-9 ]+',"",text)

In [128]:
text

' PROJECT 21  THE BEST GAMEFI PROJECT ON BSCMultistage deflation tool  PRO21Multifeature NFTApp on AppStore and Google PlayPresale in AugustP21 IS GOING TO THE MOONWhitelist TG Project21 P21 Gamefi'

In [129]:
twe_lis[1]

'Need Sick Character artâ\x9d“#art #artist #Artists #animation #illustrators #illustrations #illustrationart #NFT #NFTGiveaways #3Danimation #3dart #conceptart #Concept #illustrators #SolanaNFTs #ArtistOnTwitter @ScrimFinder @BlazedRTs @SGH_RTs @smallstreamhype @ShoutGamers https://t.co/Nf3Egckt78'

In [130]:
#function to perform all the cleaning
def cleanText(text):
    text = re.sub("https?:\/\/.*[\r\n]*", "", text)
    text = re.sub("@\S+", "", text)
    text = re.sub("#", "", text)
    text = text.replace('\n',"")
    text = re.sub(r'[^a-zA-Z ]+',"",text)
    text = text.lower()
    text = text.strip()
    return text

In [131]:
cleanText(twe_lis[1])

'need sick character artart artist artists animation illustrators illustrations illustrationart nft nftgiveaways danimation dart conceptart concept illustrators solananfts artistontwitter'

In [132]:
for i in dff['tweet_text']:
    i=cleanText(i)
#(dff['tweet_text'])

In [133]:
dff.head()

,id,tweet_text,tweet_created_at,tweet_intent
0,1212762,@crypto_brody @eCoLoGy1990 @MoonrunnersNFT @It...,2022-08-06T16:56:36.000Z,Community
1,1212763,Need Sick Character artâ“#art #artist #Artist...,2022-08-06T16:56:36.000Z,Giveaway
2,1212765,@The_Hulk_NFT @INagotchiNFT @Tesla @killabears...,2022-08-06T16:56:35.000Z,Appreciation
3,1212766,@CryptoBatzNFT @DarekBTW The first project in ...,2022-08-06T16:56:35.000Z,Community
4,1212767,@sashadysonn The first project in crypto with ...,2022-08-06T16:56:34.000Z,Community


In [134]:
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [135]:
# removing stopwords
text = " ".join([word for word in cleanText(twe_lis[1]).split() if word not in stop_words])
print(text)

need sick character artart artist artists animation illustrators illustrations illustrationart nft nftgiveaways danimation dart conceptart concept illustrators solananfts artistontwitter


In [136]:
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

#Label encoding the intent column
dff['tweet_intent']= label_encoder.fit_transform(dff['tweet_intent'])
dff['tweet_intent'].unique()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


array([1, 3, 0, 6, 7, 8, 2, 4, 5])

In [137]:
label_encoder.transform(['Community'])

array([1])

In [138]:
dff['tweet_intent'].value_counts()

1    41510
3    21041
0    16451
7     8320
6     5107
2     3049
8      477
4      276
5      133
Name: tweet_intent, dtype: int64

In [139]:
#creating a new df for the tweets and tweet intent
clean_tweet=[]
for i in twe_lis:
    clean_tweet.append(cleanText(i))
tweet_intent=[]
for i in dff['tweet_intent']:
    tweet_intent.append(i)

In [140]:
df3 = pd.DataFrame(list(zip(clean_tweet, tweet_intent)),
               columns =['Tweet', 'Label'])

In [141]:
df3.head(6)

,Tweet,Label
0,a new cryptocurrency was born by fans of the c...,1
1,need sick character artart artist artists anim...,3
2,great choice on a tesla good luck,0
3,the first project in crypto with move to earn ...,1
4,the first project in crypto with move to earn ...,1
5,just registered for the saphire on,6


In [142]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [143]:
import tensorflow as tf
from transformers import BertTokenizer

In [144]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [145]:
token = tokenizer.encode_plus(
    df3['Tweet'].iloc[0], 
    max_length=256, 
    truncation=True, 
    padding='max_length', 
    add_special_tokens=True,
    return_tensors='tf'
)

In [146]:
import numpy as np
X_input_ids = np.zeros((len(df3), 256))
X_attn_masks = np.zeros((len(df3), 256))

In [147]:
from tqdm.auto import tqdm
def generate_training_data(df, ids, masks, tokenizer):
    for i, text in tqdm(enumerate(df3['Tweet'])):
        tokenized_text = tokenizer.encode_plus(
            text,
            max_length=256, 
            truncation=True, 
            padding='max_length', 
            add_special_tokens=True,
            return_tensors='tf'
        )
        ids[i, :] = tokenized_text.input_ids
        masks[i, :] = tokenized_text.attention_mask
    return ids, masks

In [148]:
X_input_ids, X_attn_masks = generate_training_data(df3, X_input_ids, X_attn_masks, tokenizer)

0it [00:00, ?it/s]

In [149]:
#ONE HOT ENCODING
labels = np.zeros((len(df3), 9))
labels.shape

(96364, 9)

In [150]:
labels[np.arange(len(df3)), df3['Label'].values] = 1 # one-hot encoded target tensor

In [151]:
labels

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [152]:
# creating a data pipeline using tensorflow dataset utility, creates batches of data for easy loading
dataset = tf.data.Dataset.from_tensor_slices((X_input_ids, X_attn_masks, labels))
dataset.take(1) # one sample data

<TakeDataset element_spec=(TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(256,), dtype=tf.float64, name=None), TensorSpec(shape=(9,), dtype=tf.float64, name=None))>

In [153]:
def SentimentDatasetMapFunction(input_ids, attn_masks, labels):
    return {
        'input_ids': input_ids,
        'attention_mask': attn_masks
    }, labels

In [154]:
dataset = dataset.map(SentimentDatasetMapFunction) # converting to required format for tensorflow dataset

In [155]:
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(256,), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(256,), dtype=tf.float64, name=None)}, TensorSpec(shape=(9,), dtype=tf.float64, name=None))>

In [156]:
dataset = dataset.shuffle(10000).batch(16, drop_remainder=True) # batch size, drop any left out tensor
dataset.take(1)

<TakeDataset element_spec=({'input_ids': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None), 'attention_mask': TensorSpec(shape=(16, 256), dtype=tf.float64, name=None)}, TensorSpec(shape=(16, 9), dtype=tf.float64, name=None))>

In [157]:
p = 0.8
train_size = int((len(df)//16)*p) 
train_size
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

In [ ]:
#USING THE PRETRAINED BERT MODEL FOR TRAINING AND FINETUNING

In [158]:
from transformers import TFBertModel
model = TFBertModel.from_pretrained('bert-base-cased') # bert base model with pretrained weights

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [159]:
# defining 2 input layers for input_ids and attn_masks
input_ids = tf.keras.layers.Input(shape=(256,), name='input_ids', dtype='int32')
attn_masks = tf.keras.layers.Input(shape=(256,), name='attention_mask', dtype='int32')

bert_embds = model.bert(input_ids, attention_mask=attn_masks)[1] # 0 -> activation layer (3D), 1 -> pooled output layer (2D)
intermediate_layer = tf.keras.layers.Dense(512, activation='relu', name='intermediate_layer')(bert_embds)
output_layer = tf.keras.layers.Dense(9, activation='softmax', name='output_layer')(intermediate_layer) # softmax -> calcs probs of classes

sentiment_model = tf.keras.Model(inputs=[input_ids, attn_masks], outputs=output_layer)
sentiment_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 256)]        0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 256)]        0           []                               
                                                                                                  
 bert (TFBertMainLayer)         TFBaseModelOutputWi  108310272   ['input_ids[0][0]',              
                                thPoolingAndCrossAt               'attention_mask[0][0]']         
                                tentions(last_hidde                                               
                                n_state=(None, 256,                                         

In [160]:
optim = tf.keras.optimizers.Adam(learning_rate=5e-5, decay=1e-6)
loss_func = tf.keras.losses.CategoricalCrossentropy()
acc = tf.keras.metrics.CategoricalAccuracy('accuracy')
sentiment_model.compile(optimizer=optim, loss=loss_func, metrics=[acc])
hist = sentiment_model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1
)
sentiment_model.save('sentiment_model')

6022/6022 [==============================] - 5015s 831ms/step - loss: 0.2147 - accuracy: 0.9399


"""" 
According to the label encodings
[1] Community
[3] Giveaway
[0] Appreciation
[6] Presale
[7] Whitelist
[8] pinksale
[2] Done
[4] Interested
[5] Launching Soon 
""""

In [1]:
#PREDICTION

In [162]:
sentiment_model = tf.keras.models.load_model('sentiment_model')

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

def prepare_data(input_text, tokenizer):
    token = tokenizer.encode_plus(
        input_text,
        max_length=256, 
        truncation=True, 
        padding='max_length', 

        add_special_tokens=True,
        return_tensors='tf'
    )
    return {
        'input_ids': tf.cast(token.input_ids, tf.float64),
        'attention_mask': tf.cast(token.attention_mask, tf.float64)
    }

def make_prediction(model, processed_data, classes=['Appreciation', 'Community', 'Done', 'Giveaway', 'Interested','Launching Soon','Presale','Whitelist','pinksale']):
    probs = model.predict(processed_data)[0]
    return classes[np.argmax(probs)]

In [115]:
#ALL THE TEST TWEETS WHOSE INTENT NEEDS TO BE PREDICTED IS CONTAINED IN THE DATAFRAME tar
tar.head()

,id,tweet_text,tweet_created_at,tweet_intent
96364,1212764,15% off ðŸ§™â€â™‚ï¸ #Wizardia NFTs - it's a ...,2022-08-06T16:56:35.000Z,NaN
96365,1212770,@asksabir007 @dao_lm Don't miss the chance to ...,2022-08-06T16:56:33.000Z,NaN
96366,1212773,This zesty faucet from @_bitcoiner is making m...,2022-08-06T16:56:33.000Z,NaN
96367,1212804,15% off ðŸ§™â€â™‚ï¸ #Wizardia NFTs - it's a ...,2022-08-06T16:56:23.000Z,NaN
96368,1212776,"@Kavaboy_NFT @tanya_4d Message me, letâ€™s tal...",2022-08-06T16:56:32.000Z,NaN


In [163]:
test_tweets=[]
for i in tar['tweet_text']:
    test_tweets.append(i)

In [164]:
clean_test_tweet=[]
for i in test_tweets:
    clean_test_tweet.append(cleanText(i))

In [ ]:
results=[]
for i in range(len(clean_test_tweet)):    
    input_text = (clean_test_tweet[i])
    processed_data = prepare_data(input_text, tokenizer)
    result = make_prediction(sentiment_model, processed_data=processed_data)
    results.append(result)
    print(f"Predicted intent: {result}")

Streaming output truncated to the last 5000 lines.
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Presale
Predicted intent: Whitelist
Predicted intent: Presale
Predicted intent: Whitelist
Predicted intent: Giveaway
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Community
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Predicted intent: Whitelist
Pr

In [ ]:
#CREATING A DATAFRAME OF ALL THE TWEETS WHOSE INTENT NEEDS TO BE PREDICTED AND THE PREDICTIONS MADE
df_pred = pd.DataFrame(list(zip(test_tweets,clean_test_tweet,results)),
               columns =['Tweet','Clean Tweet','Label'])